
<br>

<div style ="width: 100%;">
<table style ="width: 100%;background-color:float:center;">
<tr>

</tr>
    
<tr>
        
<td>
<div style="font-size: 21px;Margin-left:-10px;">
<b>Esquema de assinatura ElGamal <b>
</div>
    <br>
    <br>
<div style="font-size: 16px;">
    
Realizado por:

Carlos Ferreira a87953

Daniel Ribeiro a87994

João Nogueira a87973
</div>
</td>
<td >
<div style="display:inline-block; float:right;Margin-right:0px;Margin-top:20px;">
<img src="https://www.ecum.uminho.pt/SiteAssets/ECUM-PT.png" width="260" height="120">
<div>
</td>
</tr>
</table>
</div>
    

# Esquema de assinatura ElGamal

No seguinte trabalho foi implementada a assinatura de ElGamal, que consiste numa tecnica baseada em propriedades algebricas da exponenciação modular com o problema do logaritmo discreto.
<br>
<br>
A chave privada de cada utilizador é usada para gerar uma assinatura digital para a mensagem. Esta assinatura pode ser verificada usando a sua chave publica.
O recetor pode verificar a origem da mensagem e pode tambem verificar se a mensagem foi ou nao alterada desde o seu envio.

## Geração de chaves

Começamos por definir a função $\textbf{keygen}$ onde geramos as chaves públicas e privadas para um nome passado por parâmetro e aplicamos o algoritmo de geração das chaves:

- Geramos um número primo á sorte com dimensões elevadas

- Calculamos uma raíz primitiva de p sendo este valor o nosso 'a'

- Encontramos um x tal que $1 \le x < p - 1$ e o máximo divisor comum entre x e p-1 tem de igualar 1

- Calculamos ${c ≡ a^{x} (mod \space p)}$

Finalmente as nossas chaves vão ser as seguintes:

- Chave pública = (p,a,c)

- Chave privada = x


In [124]:
def keygen(nome): 
    keys = {}
    p = random_prime(2^257 ,2^256)
    a = primitive_root(p)
    x = randint(1,p-2)

    while (gcd(x,p-1) != 1):
        x = randint (1,p-2)

    c = power_mod(a,x,p)
    keys[nome] = {}
    keys[nome]['public_key'] = (p,a,c)
    keys[nome]['private_key'] = x
    return keys


In [66]:
#função auxiliar para impressão bonita do dicionário
def prettyPrint(d, indent=0):
      for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):
             prettyPrint(value, indent+1)
        else:
             print('\t' * (indent+1) + str(value))

keys = keygen('Joao')
prettyPrint(keys)

Joao
	public_key
		(219598960246022047631070004374750985871339110655288597228881898077823373956563, 2, 205177880114982115612390367654537061481379965664333615073299586074419790318876)
	private_key
		4675333437489562474321285535213297625314903387781075378913563126394002043645


## Assinatura

Na nossa função de assinatura é onde assinamos a mensagem, vai ter como argumentos o dicionário das chaves e o nome de quem vai enviar a mensagem.



Começamos por fazer um hash na mensagem m e de seguida, geramos um k tal que: 
$${\displaystyle 0<k<p-2}$$
$${\displaystyle e}$$
$${\displaystyle gcd(k,p-1)=1}$$

Após isso iremos gerar um r tal que: 
$${\displaystyle r=a^kmod(p-1)}$$

Por fim iremos gerar um s tal que: 
$${\displaystyle s=H(m)-x\cdot r \cdot k^{-1}mod(p-1)}$$
Caso s=0, a função assinatura volta a executar 

In [96]:
import hashlib
def signature(sender,mensagem): 
    keys = keygen(sender)
    HH = hashlib.sha256(mensagem).hexdigest()
    HHH = ZZ('0x'+HH)
    x = keys[sender]['private_key']
    p = keys[sender]['public_key'][0]
    a = keys[sender]['public_key'][1]
    k = randint(0,p-1)
    
    while gcd(k,p-1)!=1: 
        k=randint(0,p-1)
    
    
    r = power_mod(a,k,p)
    s = power_mod((HHH - x*r)*(1/k),1,p-1)

    if s == 0: 
        signature(keys, sender, mensagem)
    return (keys,mensagem,r,s)

## Verifica assinatura

Para verificar uma assinatura (r,s) de uma mensagem m precisamos que ela valide as seguintes condições:

c1 = $0 < r < p$

c2 = $0 < s < p-1$

Caso uma destas não for válida a mensagem é logo rejeitada.

Após isto computamos:

$v1 ≡ {a^{H(m)}} (mod \space p)$   onde o H(m) é o hash da mensagem m

$v2 ≡ c^{r} \cdot r^{s} (mod \space p)$

E se

c3 = $v1 = v2$

for verdade todas as 3 condições são satisfeitas a assinatura é válida caso uma destas condições não for satisfeita, rejeitamos a assinatura.



In [146]:
 def verifica(keys,r,s,mensagem,sender):
    
    p = keys[sender]['public_key'][0]
    a = keys[sender]['public_key'][1]
    c = keys[sender]['public_key'][2]
    
    print("p = ",p)
    print("a = ",a)
    print("c = ",c)

    
    HH = hashlib.sha256(mensagem).hexdigest()
    HHH = ZZ('0x'+HH)
    
    c1 = (r > 0 and r < p)
    if not c1:
        return False
    
    c2 = (s > 0 and s < p-1) 
    if not c2:
        return False
    
    v1 = power_mod(a,HHH,p)
    print("v1 = ",v1)
    v2 = mod(power_mod(c,r,p) * power_mod(r,s,p) , p)
    print("v2 = ",v2)
    
    c3 = v1 == v2
    
    
    print("r = ",r)
    print("s = ",s)
    
    return c3
        

## Exemplos

Com uns pequenos exemplos vamos testar o nosso sistema:

In [153]:
keys, m , r ,s = signature ('Daniel', b"Ola Henrique")
verifica(keys , r , s, m,'Daniel')

p =  322066867625995949651802024307832585989982912762994038985418410510542277037
a =  2
c =  299800413030344079346475731895212270283485172857878589866316567565134918041
v1 =  45323266731041446061595342794447613463033154671369080909180530772500710563
v2 =  45323266731041446061595342794447613463033154671369080909180530772500710563
r =  126069318053276631833079698317830564794358672259759660812185766945926028136
s =  22693853395904618277955249520017208851554936454775904289306854357770311525


True

In [147]:
keys, m , r ,s = signature ('Joao', b"Ola Daniel")
verifica(keys , r , s, m,'Joao')

p =  11026900836479145608238978157666650549682396748252659100010975400028585308987
a =  2
c =  3281677727992107301629223622146081403850480284761787124754214117830560073228
v1 =  7371396516431643636727253463630624268186817973709143350788214782167490794476
v2 =  7371396516431643636727253463630624268186817973709143350788214782167490794476
r =  2323218945885483216033600166767389254019625863675279925973287324660341211117
s =  2034562847686403902724285901888985529320323565312707633154153834454749310366


True

In [150]:
keys, m , r ,s = signature ('Joao', b"Ola Henrique")
verifica(keys , r , s, b"Ola Joao",'Joao')

p =  218634623098503492693233197324713247720131145773260543417317257422890714167503
a =  5
c =  152332420519541666115225772978643933221302172211025947509192045764924348160154
v1 =  105198032881613611275633178823678768038510776157497993507719783202820420662526
v2 =  44799378430074846678848887985342303685128457959752068762574474844669286096023
r =  122439669261371935382996665076687343128875658658927084544773828741876812607288
s =  114031886746554783367892027842476509655174135132092780865002364393292133308753


False